In [ ]:
#this program pulls in each individual POS file that gets sent to us each month from our distributors
#frustratingly, they are in all different formats - this program extracts the data we need from each one and concatenates into one standard, uniform table
#various data cleaning is performed, including flagging new customer names - allows for name standardization/consolidation that in turn improves later analysis
#commissions to sales reps are assigned based on a list of rules
#file is uploaded to SQL database for company visibility and reporting to parent company Amphenol

#ideally this could be a SQL stored procedure - but the reality of the situation makes this Python script more feasible
#there remain some manual aspects to this - but overall, this project has been a big win for the company and created value in numerous areas
#several members of the team have had numerous hours reduced from their monthly workload, freeing them up for other tasks
#cleaner and more accurate data than before
#speedier and more efficient reporting

In [ ]:
#import needed packages
import pandas as pd
import urllib.request
from pprint import pprint
from html_table_parser.parser import HTMLTableParser

In [ ]:
def url_get_contents(url):
    #Opens a website and read its
    #binary contents (HTTP Response Body)

    #making request to the website
    req = urllib.request.Request(url=url)
    f = urllib.request.urlopen(req)

    #reading contents of the website
    return f.read()

In [ ]:
#defining the html contents of a URL; this gives an up-to-date currency conversion table needed for some of the international files

xhtml = url_get_contents('https://www.xe.com/currencytables/?from=USD&date=2022-09-30#table-section').decode('utf-8') 
###########################################################################################################################
#change date in above url to last day of POS month for most current conversion table
###########################################################################################################################

#Defining the HTMLTableParser object
p = HTMLTableParser()

#feeding the html contents in the
#HTMLTableParser object
p.feed(xhtml)

#Now finally obtaining the data of
#the table required
conversion=list((p.tables[0]))
conversion1=pd.DataFrame(conversion)
conversion1.columns =['Currency', 'Name', 'USD to Currency', 'Currency to USD']
conversion1 = conversion1.iloc[1: , :]
conversion1['Currency to USD']=conversion1['Currency to USD'].str.replace(',','')
conversion1['Currency to USD']=pd.to_numeric(conversion1['Currency to USD'])
conversion1.loc[conversion1['Currency'] == 'CNY', 'Currency'] = 'RMB'
conversion1.head(11)

In [ ]:
#pull in individual distributor POS files; comment out any that had no sales for a given month
aepetsche=pd.read_csv('AEPetsche.csv',header=3).dropna(subset=['Quantity            '])
applied=pd.read_csv('Applied.csv',header=3).iloc[:-2,:]
arrow=pd.read_csv('Arrow.csv')
avnet=pd.read_csv('Avnet.csv',header=1).iloc[:-1,:]
bjg=pd.read_csv('BJG.csv')
digikey=pd.read_csv('Digikey.csv').iloc[:-1,:] 
electro=pd.read_csv('Electro.csv')
farnell=pd.read_csv('Farnell.csv',encoding='latin')
heilind=pd.read_csv('Heilind.csv',header=None) #manually combine all relevant files in Excel- delete first row and last row of all
heilindapac=pd.read_csv('HeilindAPAC.csv',header=None) #manually combine all relevant files in Excel- delete first row and last row of all
heilindeurope=pd.read_csv('HeilindEurope.csv',header=None) #manually combine all relevant files in Excel- delete first row and last row of all
master=pd.read_csv('Master.csv').iloc[:-1,:]
mci=pd.read_csv('MCI.csv')
mouser=pd.read_csv('Mouser.csv',header=1).iloc[:-1,:]
newark=pd.read_csv('Newark.csv').drop(0)
pei=pd.read_csv('PEI.csv').iloc[:-1,:]
peieurope=pd.read_csv('PEIEurope.csv').iloc[:-1,:]
richardson=pd.read_csv('Richardson.csv',header=2).iloc[:-4,:] #fixed width when converting to columns in Excel; manually edit column widths/breaks; be careful and check head/tail
#spirit=pd.read_csv('Spirit.csv')
swift=pd.read_csv('Swift.csv',header=2)
ttieurope=pd.read_csv('TTIEurope.csv').iloc[:-1,:]
tti=pd.read_csv('TTI.csv').iloc[:-1,:]

In [ ]:
#the next several blocks of code pull in the desired data from each distributor file
#also show the first and last 5 rows of each file as a way to eyeball anomalies

In [ ]:
aepetsche['Date']=pd.to_datetime(aepetsche['Date Shipped'])
aepetsche['DistributorID']=12139
aepetsche['Distributor']='A.E. PETSCHE'
aepetsche['CustomerID']=aepetsche.iloc[:,0]
aepetsche['Customer']=aepetsche.iloc[:,1]
aepetsche['Purchased']=''
aepetsche['Address']=aepetsche['Address']
aepetsche['City']=aepetsche['City']
aepetsche['State']=aepetsche['State']
aepetsche['ZIP']=aepetsche['Zip Code']
aepetsche['Country']=aepetsche['Country']
aepetsche['Item Number']=aepetsche['MIL PN']
aepetsche['Quantity']=aepetsche.iloc[:,9]
aepetsche['Acquisition Price']=aepetsche.iloc[:,12]
aepetsche['Resale Price']=aepetsche.iloc[:,10]
aepetsche=aepetsche[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
aepetsche.head()

In [ ]:
aepetsche.tail()

In [ ]:
applied['Date']=pd.to_datetime('09/01/22') #change to first day of POS month
applied['DistributorID']=10013
applied['Distributor']='APPLIED'
applied['CustomerID']=''
applied['Customer']=applied['CUSTOMER']
applied['Purchased']=''
applied['Address']=''
applied['City']=applied['CITY']
applied['State']=applied['STATE']
applied['ZIP']=applied['ZIP CODE']
applied['Country']='US' #check
applied['Item Number']=applied['P/N']
applied['Quantity']=applied['QTY']
applied['Acquisition Price']=applied.iloc[:,9]/applied['QTY']
applied['Resale Price']=applied.iloc[:,7]
applied=applied[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
applied.head()

In [ ]:
applied.tail()

In [ ]:
arrow['INV-DATE']=arrow['INV-DATE'].astype(str)
arrow['Date']=arrow['INV-DATE'].str[4:6]+'/'+arrow['INV-DATE'].str[6:8]+'/'+arrow['INV-DATE'].str[0:4]
arrow['Date']=pd.to_datetime(arrow['Date'])
arrow['DistributorID']=11459
arrow['Distributor']='ARROW'
arrow['CustomerID']=arrow['CUST-ID']
arrow['Customer']=arrow['SHIP-TO-NAME']
arrow['Purchased']=''
arrow['Address']=arrow['SHIP_TO_ADDR']
arrow['City']=arrow['SHIP_TO_CITY']
arrow['State']=arrow['SHIP-TO-ST']
arrow['ZIP']=arrow['SHIP-TO-ZIP']
arrow['Country']=arrow['SHIP_TO_CNTRY']
arrow['Item Number']=arrow['PART-NBR']
arrow['Quantity']=arrow['QTY']
arrow['Acquisition Price']=arrow['UNIT-COST']
arrow['Resale Price']=arrow['RESALE-PRICE']
arrow=arrow[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
arrow.head()

In [ ]:
arrow.tail()

In [ ]:
avnet['InvoiceDate']=avnet['InvoiceDate'].astype(str)
avnet['Date']=avnet['InvoiceDate'].str[4:6]+'/'+avnet['InvoiceDate'].str[6:8]+'/'+avnet['InvoiceDate'].str[0:4]
avnet['Date']=pd.to_datetime(avnet['Date'])
avnet['DistributorID']=12292
avnet['Distributor']='AVNET'
avnet['CustomerID']=avnet['CustomerId']
avnet['Customer']=avnet['ShipToCustomerName']
avnet['Purchased']=''
avnet['Address']=avnet['ShipToAddress1']
avnet['City']=avnet['ShipToCity']
avnet['State']=avnet['ShipToState']
avnet['ZIP']=avnet['ShipToZipCode']
avnet['Country']=avnet['ShipToCountry']
avnet['Item Number']=avnet['VendorPartNumber']
avnet['Quantity']=avnet['QuantityShipped']
avnet['Acquisition Price']=avnet['UnitCost']
avnet['Resale Price']=avnet['ResaleAmount']
avnet=avnet[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
avnet.head()

In [ ]:
avnet.tail()

In [ ]:
bjg['Date']=pd.to_datetime(bjg['Invoice Date'])
bjg['DistributorID']=10922
bjg['Distributor']='BJG, INC.'
bjg['CustomerID']=''
bjg['Customer']=bjg['Customer Name']
bjg['Purchased']=''
bjg['Address']=''
bjg['City']=bjg['City']
bjg['State']=bjg['State']
bjg['ZIP']=bjg['Zip']
bjg['Country']='US' #check
bjg['Item Number']=bjg['Part']
bjg['Quantity']=bjg['Qty Shipped']
bjg['Acquisition Price']=bjg['Unit Cost']
bjg['Resale Price']=bjg['Unit Price']
bjg=bjg[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
bjg.head()

In [ ]:
bjg.tail()

In [ ]:
digikey['INV-DATE']=digikey.iloc[:,12].astype(str)
digikey['Date']=digikey['INV-DATE'].str[4:6]+'/'+digikey['INV-DATE'].str[6:8]+'/'+digikey['INV-DATE'].str[0:4]
digikey['Date']=pd.to_datetime(digikey['Date'])
digikey['DistributorID']=12261
digikey['Distributor']='DIGI-KEY'
digikey['CustomerID']=digikey.iloc[:,2]
digikey['Customer']=digikey.iloc[:,3]
digikey['Purchased']=''
digikey['Address']=digikey.iloc[:,4]
digikey['City']=digikey.iloc[:,5]
digikey['State']=digikey.iloc[:,6]
digikey['ZIP']=digikey.iloc[:,7]
digikey['Country']=digikey.iloc[:,19]
digikey['Item Number']=digikey.iloc[:,8]
digikey['Quantity']=digikey.iloc[:,9]
digikey['Acquisition Price']=digikey.iloc[:,13]
digikey['Resale Price']=digikey.iloc[:,14]
digikey=digikey[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
digikey.head()

In [ ]:
digikey.tail()

In [ ]:
electro['Date']=pd.to_datetime(electro['Ship Date'])
electro['DistributorID']=12418
electro['Distributor']='ELECTRO ENTERPRISES'
electro['CustomerID']=''
electro['Customer']=electro['Customer Name']
electro['Purchased']=''
electro['Address']=''
electro['City']=electro['Ship To City']
electro['State']=electro['Ship To State']
electro['ZIP']=electro['Ship To Zip']
electro['Country']=electro['Ship To Country']
electro['Item Number']=electro['Part Number']
electro['Quantity']=electro['Qty']
electro['Acquisition Price']=electro['Unit Cost']
electro['Resale Price']=electro['Unit Resale']
electro=electro[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
electro.head()

In [ ]:
electro.tail()

In [ ]:
farnell=pd.merge(farnell, conversion1, left_on='Currency',right_on='Currency',how='inner')
farnell['INV-DATE']=farnell['Date'].astype(str)
farnell['Date']=farnell['INV-DATE'].str[4:6]+'/'+farnell['INV-DATE'].str[6:8]+'/'+farnell['INV-DATE'].str[0:4]
farnell['Date']=pd.to_datetime(farnell['Date'])
farnell['DistributorID']=12310
farnell['Distributor']='FARNELL'
farnell['CustomerID']=''
farnell['Customer']=farnell['Customer Name']
farnell['Purchased']=''
farnell['Address']=''
farnell['City']=''
farnell['State']=''
farnell['ZIP']=farnell['Post Code']
farnell['Country']=farnell['Country Code']
farnell['Item Number']=farnell['MFG Product Code']
farnell['Quantity']=farnell['Qty']
farnell['Acquisition Price']=farnell['Unit Cost System']*farnell['Currency to USD']
farnell['Resale Price']=farnell['Unit Price Currency']*farnell['Currency to USD']
farnell=farnell[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
farnell.head()

In [ ]:
farnell.tail()

In [ ]:
heilind['INV-DATE']=heilind.iloc[:,12].astype(str)
heilind['Date']=heilind['INV-DATE'].str[4:6]+'/'+heilind['INV-DATE'].str[6:8]+'/'+heilind['INV-DATE'].str[0:4]
heilind['Date']=pd.to_datetime(heilind['Date'])
heilind['DistributorID']=12296
heilind['Distributor']='HEILIND'
heilind['CustomerID']=heilind.iloc[:,2]
heilind['Customer']=heilind.iloc[:,3]
heilind['Purchased']=''
heilind['Address']=heilind.iloc[:,4]
heilind['City']=heilind.iloc[:,5]
heilind['State']=heilind.iloc[:,6]
heilind['ZIP']=heilind.iloc[:,7]
heilind['Country']=heilind.iloc[:,19]
heilind['Item Number']=heilind.iloc[:,8]
heilind['Quantity']=heilind.iloc[:,9]
heilind['Acquisition Price']=heilind.iloc[:,13]
heilind['Resale Price']=heilind.iloc[:,14]
heilind=heilind[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
heilind.head()

In [ ]:
heilind.tail()

In [ ]:
heilindapac['INV-DATE']=heilindapac.iloc[:,12].astype(str)
heilindapac['Date']=heilindapac['INV-DATE'].str[4:6]+'/'+heilindapac['INV-DATE'].str[6:8]+'/'+heilindapac['INV-DATE'].str[0:4]
heilindapac['Date']=pd.to_datetime(heilindapac['Date'])
heilindapac['DistributorID']=12296
heilindapac['Distributor']='HEILIND - APAC'
heilindapac['CustomerID']=heilindapac.iloc[:,2]
heilindapac['Customer']=heilindapac.iloc[:,3]
heilindapac['Purchased']=''
heilindapac['Address']=heilindapac.iloc[:,4]
heilindapac['City']=heilindapac.iloc[:,5]
heilindapac['State']=heilindapac.iloc[:,6]
heilindapac['ZIP']=heilindapac.iloc[:,7]
heilindapac['Country']=heilindapac.iloc[:,19]
heilindapac['Item Number']=heilindapac.iloc[:,8]
heilindapac['Quantity']=heilindapac.iloc[:,9]
heilindapac['Acquisition Price']=heilindapac.iloc[:,13]
heilindapac['Resale Price']=heilindapac.iloc[:,14]
heilindapac=heilindapac[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
heilindapac.head()

In [ ]:
heilindapac.tail()

In [ ]:
heilindeurope['INV-DATE']=heilindeurope.iloc[:,12].astype(str)
heilindeurope['Date']=heilindeurope['INV-DATE'].str[4:6]+'/'+heilindeurope['INV-DATE'].str[6:8]+'/'+heilindeurope['INV-DATE'].str[0:4]
heilindeurope['Date']=pd.to_datetime(heilindeurope['Date'])
heilindeurope['DistributorID']=12296
heilindeurope['Distributor']='HEILIND - EUROPE'
heilindeurope['CustomerID']=heilindeurope.iloc[:,2]
heilindeurope['Customer']=heilindeurope.iloc[:,3]
heilindeurope['Purchased']=''
heilindeurope['Address']=heilindeurope.iloc[:,4]
heilindeurope['City']=heilindeurope.iloc[:,5]
heilindeurope['State']=heilindeurope.iloc[:,6]
heilindeurope['ZIP']=heilindeurope.iloc[:,7]
heilindeurope['Country']=heilindeurope.iloc[:,19]
heilindeurope['Item Number']=heilindeurope.iloc[:,8]
heilindeurope['Quantity']=heilindeurope.iloc[:,9]
heilindeurope['Acquisition Price']=heilindeurope.iloc[:,13]
heilindeurope['Resale Price']=heilindeurope.iloc[:,14]
heilindeurope=heilindeurope[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
heilindeurope.head()

In [ ]:
heilindeurope.tail()

In [ ]:
master['INV-DATE']=master.iloc[:,15].astype(str)
master['Date']=master['INV-DATE'].str[4:6]+'/'+master['INV-DATE'].str[6:8]+'/'+master['INV-DATE'].str[0:4]
master['Date']=pd.to_datetime(master['Date'])
master['DistributorID']=12483
master['Distributor']='MASTER ELECTRONICS'
master['CustomerID']=master.iloc[:,12]
master['Customer']=master.iloc[:,13]
master['Purchased']=''
master['Address']=''
master['City']=master.iloc[:,2]
master['State']=master.iloc[:,3]
master['ZIP']=master.iloc[:,4]
master['Country']=master.iloc[:,25]
master['Item Number']=master.iloc[:,5]
master['Quantity']=master.iloc[:,6]
master['Acquisition Price']=master.iloc[:,9]
master['Resale Price']=master.iloc[:,10]
master=master[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
master.head()

In [ ]:
master.tail()

In [ ]:
mci['Date']=pd.to_datetime(mci['Invoice Date'])
mci['DistributorID']=11436
mci['Distributor']='MCI'
mci['CustomerID']=mci['Customer ID']
mci['Customer']=mci['Customer Name']
mci['Purchased']=''
mci['Address']=''
mci['City']=mci['City']
mci['State']=mci['State']
mci['ZIP']=mci['Zip']
mci['Country']=mci['Country']
mci['Item Number']=mci['Part Number']
mci['Quantity']=mci['Qty']
mci['Acquisition Price']=mci['Unit Cost']
mci['Resale Price']=mci['Resale cost']
mci=mci[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
mci.head()

In [ ]:
mci.tail()

In [ ]:
mouser['InvoiceDate']=mouser['DATE'].astype(str)
mouser['Date']=mouser['InvoiceDate'].str[4:6]+'/'+mouser['InvoiceDate'].str[6:8]+'/'+mouser['InvoiceDate'].str[0:4]
mouser['Date']=pd.to_datetime(mouser['Date'])
mouser['DistributorID']=12093
mouser['Distributor']='MOUSER'
mouser['CustomerID']=mouser['ACCT.']
mouser['Customer']=mouser['NAME']
mouser['Purchased']=mouser['BRANCH']
mouser['Address']=''
mouser['City']=mouser['CITY']
mouser['State']=mouser['STATE']
mouser['ZIP']=mouser['ZIP']
mouser['Country']=mouser['COUNTRY CODE']
mouser['Item Number']=mouser['PART NO']
mouser['Quantity']=mouser['QTY']
mouser['Acquisition Price']=mouser['COST']
mouser['Resale Price']=mouser['PRICE']
mouser=mouser[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
mouser.head()

In [ ]:
mouser.tail()

In [ ]:
newark['INV-DATE']=newark['Ship Date'].astype(str)
newark['Date']=newark['INV-DATE'].str[4:6]+'/'+newark['INV-DATE'].str[6:8]+'/'+newark['INV-DATE'].str[0:4]
newark['Date']=pd.to_datetime(newark['Date'])
newark['DistributorID']=12303
newark['Distributor']='NEWARK'
newark['CustomerID']=newark['Ship-to Customer ID']
newark['Customer']=newark.iloc[:,2]
newark['Purchased']=''
newark['Address']=newark.iloc[:,25]
newark['City']=newark.iloc[:,3]
newark['State']=newark.iloc[:,22]
newark['ZIP']=newark.iloc[:,4]
newark['Country']=newark.iloc[:,23]
newark['Item Number']=newark.iloc[:,5]
newark['Quantity']=newark.iloc[:,6]
newark['Acquisition Price']=newark.iloc[:,10]
newark['Resale Price']=newark.iloc[:,11]
newark=newark[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
newark.head()

In [ ]:
newark.tail()

In [ ]:
pei=pd.merge(pei, conversion1, left_on=pei.iloc[:,21],right_on='Currency',how='inner')
pei['INV-DATE']=pei.iloc[:,10].astype(str)
pei['Date']=pei['INV-DATE'].str[4:6]+'/'+pei['INV-DATE'].str[6:8]+'/'+pei['INV-DATE'].str[0:4]
pei['Date']=pd.to_datetime(pei['Date'])
pei['DistributorID']=12398
pei['Distributor']='PEI'
pei['CustomerID']=pei.iloc[:,14]
pei['Customer']=pei.iloc[:,15]
pei['Purchased']=''
pei['Address']=pei.iloc[:,26]
pei['City']=pei.iloc[:,3]
pei['State']=pei.iloc[:,4]
pei['ZIP']=pei.iloc[:,5]
pei['Country']=pei.iloc[:,19]
pei['Item Number']=pei.iloc[:,6]
pei['Quantity']=pei.iloc[:,7]
pei['Acquisition Price']=pei.iloc[:,11]*pei['Currency to USD']
pei['Resale Price']=pei.iloc[:,12]*pei['Currency to USD']
pei=pei[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
pei.head()

In [ ]:
pei.tail()

In [ ]:
#add %%script false to any distributor block that was missing for the month; this will skip it when run
#comment out the respective head and tail

In [ ]:
%%script false
peieurope=pd.merge(peieurope, conversion1, left_on=peieurope.iloc[:,21],right_on='Currency',how='inner')
peieurope['INV-DATE']=peieurope.iloc[:,10].astype(str)
peieurope['Date']=peieurope['INV-DATE'].str[4:6]+'/'+peieurope['INV-DATE'].str[6:8]+'/'+peieurope['INV-DATE'].str[0:4]
peieurope['Date']=pd.to_datetime(peieurope['Date'])
peieurope['DistributorID']=12385
peieurope['Distributor']='PEI - EUROPE'
peieurope['CustomerID']=peieurope.iloc[:,14]
peieurope['Customer']=peieurope.iloc[:,15]
peieurope['Purchased']=''
peieurope['Address']=peieurope.iloc[:,26]
peieurope['City']=peieurope.iloc[:,3]
peieurope['State']=peieurope.iloc[:,4]
peieurope['ZIP']=peieurope.iloc[:,5]
peieurope['Country']=peieurope.iloc[:,19]
peieurope['Item Number']=peieurope.iloc[:,6]
peieurope['Quantity']=peieurope.iloc[:,7]
peieurope['Acquisition Price']=peieurope.iloc[:,11]*peieurope['Currency to USD']
peieurope['Resale Price']=peieurope.iloc[:,12]*peieurope['Currency to USD']
peieurope=peieurope[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]
peieurope=peieurope[~peieurope['Customer'].str.startswith('PEI')]

In [ ]:
#peieurope.head()

In [ ]:
#peieurope.tail()

In [ ]:
richardson['Date']=pd.to_datetime(richardson.iloc[:,7])
richardson['DistributorID']=11550
richardson['Distributor']='RICHARDSON'
richardson['CustomerID']=richardson.iloc[:,0]
richardson['Customer']=richardson.iloc[:,1]
richardson['Purchased']=''
richardson['Address']=''
richardson['City']=richardson.iloc[:,2]
richardson['State']=richardson.iloc[:,3]
richardson['ZIP']=richardson.iloc[:,4]
richardson['Country']=richardson.iloc[:,5]
richardson['Country']=richardson['Country'].fillna('US') #check
richardson['Item Number']=richardson.iloc[:,8]
richardson['Quantity']=richardson.iloc[:,9]
richardson['Acquisition Price']=richardson.iloc[:,10]
richardson['Resale Price']=richardson.iloc[:,12]
richardson=richardson[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
richardson.head()

In [ ]:
richardson.tail()

In [ ]:
swift['Date']=pd.to_datetime(swift['INV DATE'])
swift['DistributorID']=10915
swift['Distributor']='SWIFT'
swift['CustomerID']=swift['CUSNO'].str.replace("'", '')
swift['Customer']=swift['CUST']
swift['Purchased']=''
swift['Address']=''
swift['City']=swift['CITY']
swift['State']=swift['STATE']
swift['ZIP']=swift['ZIP'].str.replace("'", '') #check
swift['Country']='US' #check
swift['Item Number']=swift['PARTNO']
swift['Quantity']=swift['QTY']
swift['Acquisition Price']=swift['COST']
swift['Resale Price']=swift['PRICE']
swift=swift[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
swift.head()

In [ ]:
swift.tail()

In [ ]:
tti['INV-DATE']=tti.iloc[:,15].astype(str)
tti['Date']=tti['INV-DATE'].str[4:6]+'/'+tti['INV-DATE'].str[6:8]+'/'+tti['INV-DATE'].str[0:4]
tti['Date']=pd.to_datetime(tti['Date'])
tti['DistributorID']=11912
tti['Distributor']='TTI, INC.'
tti['CustomerID']=tti.iloc[:,2]
tti['Customer']=tti.iloc[:,3]
tti['Purchased']=''
tti['Address']=tti.iloc[:,5]
tti['City']=tti.iloc[:,6]
tti['State']=tti.iloc[:,7]
tti['ZIP']=tti.iloc[:,8]
tti['Country']=tti.iloc[:,22]
tti['Item Number']=tti.iloc[:,9]
tti['Quantity']=tti.iloc[:,11]
tti['Acquisition Price']=tti.iloc[:,16]
tti['Resale Price']=tti.iloc[:,17]
tti=tti[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
tti.head()

In [ ]:
tti.tail()

In [ ]:
ttieurope=pd.merge(ttieurope, conversion1, left_on=ttieurope.iloc[:,40],right_on='Currency',how='inner')
ttieurope['INV-DATE']=ttieurope.iloc[:,15].astype(str)
ttieurope['Date']=ttieurope['INV-DATE'].str[4:6]+'/'+ttieurope['INV-DATE'].str[6:8]+'/'+ttieurope['INV-DATE'].str[0:4]
ttieurope['Date']=pd.to_datetime(ttieurope['Date'])
ttieurope['DistributorID']=11912
ttieurope['Distributor']='TTI - EUROPE'
ttieurope['CustomerID']=ttieurope.iloc[:,2]
ttieurope['Customer']=ttieurope.iloc[:,3]
ttieurope['Purchased']=''
ttieurope['Address']=ttieurope.iloc[:,5]
ttieurope['City']=''
ttieurope['State']=ttieurope.iloc[:,7]
ttieurope['ZIP']=ttieurope.iloc[:,6]
ttieurope['Country']=ttieurope.iloc[:,22]
ttieurope['Item Number']=ttieurope.iloc[:,9]
ttieurope['Quantity']=ttieurope.iloc[:,11]
ttieurope['Acquisition Price']=ttieurope.iloc[:,16]*ttieurope['Currency to USD']
ttieurope['Resale Price']=ttieurope.iloc[:,17]*ttieurope['Currency to USD']
ttieurope=ttieurope[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price']]

In [ ]:
ttieurope.head()

In [ ]:
ttieurope.tail()

In [ ]:
#concatenate data from all files; delete any that were missing
pos=pd.concat([aepetsche, applied, arrow, avnet, bjg, digikey, electro, farnell, heilind, heilindapac, heilindeurope, master, mci, mouser, newark, pei, richardson, swift, tti, ttieurope], axis=0)
len(pos)

In [ ]:
#various cleaning
pos=pos.reset_index()
pos['Date']=pd.to_datetime(pos['Date']).dt.date
pos['Customer']=pos['Customer'].str.replace(r'\s+', ' ', regex=True).str.strip().str.upper()
pos['City']=pos['City'].str.replace(r'\s+', ' ', regex=True).str.strip().str.upper().str.replace('\d+', '').str.replace('-', ' ').str.replace(r"\(.*\)","")
pos['State']=pos['State'].str.strip().str.upper()
pos['Country']=pos['Country'].str.strip().str.upper()
pos['Item Number']=pos['Item Number'].str.strip().str.replace('-B','')
pos['Quantity']=pos['Quantity'].astype(str).str.strip().str.replace(',','').str.replace('(','-').str.replace(')','')
pos['Quantity']=pd.to_numeric(pos['Quantity'])
pos['Acquisition Price']=pos['Acquisition Price'].astype(str).str.strip().str.replace(',','').str.replace('$','').str.replace('(','-').str.replace(')','').str.replace('nan','0')
pos['Acquisition Price']=pd.to_numeric(pos['Acquisition Price'])
pos['Resale Price']=pos['Resale Price'].astype(str).str.strip().str.replace(',','').str.replace('$','').str.replace('(','-').str.replace(')','')
pos['Resale Price']=pd.to_numeric(pos['Resale Price'])

In [ ]:
#view each distributor's POS $
pos['Total Resale']=pos['Quantity']*pos['Resale Price']
pos.groupby(['Distributor'])['Total Resale'].sum().reset_index

In [ ]:
#view total POS
pos['Total Resale'].sum()

In [ ]:
#drop total resale column - only using it for a quick view
pos=pos.drop(['Total Resale'], axis = 1)

In [ ]:
#customer name standardization - distributors don't have a uniform way of reporting customer names
#for better analysis on our end, we need to edit names that come in each month
#the code in this cell identifies 'new' customers that haven't appeared before (using a local file of historical POS to check)
master=pd.read_csv('POS.csv',encoding='latin')
pos1=master['Customer Name (Standardized)'].unique()
pos1=pd.DataFrame(pos1)
pos1.columns = ['Existing']
new=pos
new=new['Customer'].unique()
new=pd.DataFrame(new)
new.columns = ['New']
merged=new.merge(pos1, how='left', left_on='New', right_on='Existing', indicator=True)
newcustomers=merged[merged['_merge']=='left_only']
cref=pd.read_csv('CRef.csv',encoding='latin')
newcustomers=pd.merge(newcustomers,cref,left_on='New',right_on='Customer Name',how='left')
newcustomers=newcustomers[newcustomers['Customer Name'].isnull()]
len(newcustomers)

In [ ]:
#newcustomers

In [ ]:
#manually scan through this list, compare to list of names in local file POS.csv and add any changes to local file CRef.csv
newcustomers.to_excel('C:/Users/chines/Downloads/NewCustomersInitialCheck.xlsx', index = False, header=True)

In [ ]:
#pos[pos['Customer'].str.startswith('V')]

In [ ]:
#local file of country names and their market region
region=pd.read_csv('MarketRegion.csv').drop_duplicates(subset=['Country Name'])
region['Country Name']=region['Country Name'].str.strip().str.upper()
region['Country Code']=region['Country Code'].str.strip().str.upper()
pos=pd.merge(pos,region,left_on='Country',right_on='Country Name',how='left')
pos['CountryFinal']=''
pos.loc[pos['Country Name'].notnull(), 'CountryFinal'] = pos['Country Code']
pos.loc[pos['Country Name'].isnull(), 'CountryFinal'] = pos.Country

In [ ]:
#local file of customer names as sent to us matched to what we decide is the standardized name
#this should now include the latest month's fixes
cref=pd.read_csv('CRef.csv', encoding='latin').drop_duplicates(subset=['Customer Name'])
cref['Customer Name']=cref['Customer Name'].str.strip().str.upper()
cref['CustomerNew']=cref['CustomerNew'].str.strip().str.upper()
pos=pd.merge(pos,cref,left_on='Customer',right_on='Customer Name',how='left')
pos['CustomerFinal']=''
pos.loc[pos['Customer Name'].notnull(), 'CustomerFinal'] = pos['CustomerNew']
pos.loc[pos['Customer Name'].isnull(), 'CustomerFinal'] = pos['Customer']

In [ ]:
pos=pos.drop(['Customer Name','CustomerNew','Country','Country Name','Country Code'], axis = 1)
pos.rename(columns={'CountryFinal':'Country'},inplace=True)
pos.rename(columns={'Customer':'Original Customer Name'},inplace=True)
pos.rename(columns={'CustomerFinal':'Customer'},inplace=True)

In [ ]:
#zip code cleaning
pos.loc[pos['Country']!='US', 'ZIP'] = '0'
pos['ZIP']=pos['ZIP'].str.strip().fillna('0')
pos.loc[pos.ZIP=='nan', 'ZIP'] = '0'
pos['ZIP']=pos['ZIP'].str.replace('-','')
pos['lzip']=pos['ZIP'].str.len()
pos.loc[pos.lzip==4, 'ZIP'] = '0'+pos.ZIP
pos.loc[pos.lzip==8, 'ZIP'] = '0'+pos.ZIP.str[0:4]
pos.loc[pos.lzip==9, 'ZIP'] = pos.ZIP.str[0:5]

In [ ]:
#tag rep codes based on territory, other rules
pos['Rep Code']=401
pos.loc[pos.State.isin(['ME','NH','VT','MA','RI','CT']), 'Rep Code'] = 548
pos.loc[(pos['State']=='CT') & (pos['ZIP'].str.startswith('06')), 'Rep Code'] = 401
pos.loc[pos.State.isin(['AZ','NM']), 'Rep Code'] = 564
pos.loc[pos.State.isin(['MD','VA','WV','DC','DE']), 'Rep Code'] = 555
pos.loc[pos.State.isin(['UT','ID','MT','CO','WY']), 'Rep Code'] = 551
pos.loc[pos.State.isin(['AS','OK','TX','LA']), 'Rep Code'] = 565
pos.loc[(pos['Customer'].isin(['RAYTHEON','DUCOMMUN'])) & (pos['Rep Code']==565), 'Rep Code'] = 401
pos['ZIP']=pd.to_numeric(pos['ZIP'], errors='coerce')
pos['ZIP']=pos['ZIP'].fillna('0')
pos.loc[pos.Country.isin(['US']), 'ZIP'] = pos['ZIP'].astype(int)
pos.loc[(pos['State']=='NY') & (pos['ZIP']>=12000), 'Rep Code'] = 566
pos.loc[(pos['State']=='NY') & (pos['ZIP'].isin([12518,12520,12543,12549,12550,12551,12552,12553,12563,12555,12575,12577,12584,12586,12729,12746,12771,12780])), 'Rep Code'] = 401
pos.loc[(pos['State']=='CA') & (pos['ZIP'].between(90001,93591,inclusive=True)), 'Rep Code'] = 567
pos.loc[(pos['State']=='CA') & (pos['ZIP'].between(93600,96099,inclusive=True)), 'Rep Code'] = 331
pos.loc[(pos['State']=='CA') & (pos['ZIP'].between(91901,92199,inclusive=True)), 'Rep Code'] = 401
pos.loc[(pos['Customer']=='NORTHROP GRUMMAN') & (pos['City'].isin(['EL SEGUNDO','EL REDONDO','REDONDO BEACH'])), 'Rep Code'] = 374
pos.loc[(pos['Customer'].str.contains('AMPHENOL')) & (pos['Country']=='US'), 'Rep Code'] = 401

pos.loc[~pos['Country'].isin(['USA','US','UNITED STATES']), 'Rep Code'] = 601
pos.loc[pos['Market Region']=='EMEA', 'Rep Code'] = 701

pos.loc[pos.Customer=='SPACE-X', 'Rep Code'] = 567
pos.loc[pos.Customer=='SIERRA NEVADA', 'Rep Code'] = 331
pos.loc[pos.Customer=='MACROLINK', 'Rep Code'] = 374
pos.loc[pos.Customer=='SV MICROWAVE', 'Rep Code'] = 401
pos.loc[pos.Customer=='FAIRVIEW MICROWAVE', 'Rep Code'] = 401
pos.loc[pos.Customer=='TELEDYNE', 'Rep Code'] = 401
pos.loc[pos.Customer=='GRAYHILL', 'Rep Code'] = 401
pos.loc[pos.Customer=='MASSACHUSETTS INSTITUTE OF TECHNOLOGY', 'Rep Code'] = 401
pos.loc[pos.Customer=='US ARMY', 'Rep Code'] = 401
pos.loc[pos.Customer=='US NAVY', 'Rep Code'] = 401
pos.loc[pos.Customer=='US AIR FORCE', 'Rep Code'] = 401
pos.loc[pos.Customer=='US COAST GUARD', 'Rep Code'] = 401
pos.loc[pos.Customer=='US GOVERNMENT', 'Rep Code'] = 401
pos.loc[pos.Customer=='US MARINES', 'Rep Code'] = 401
pos.loc[pos.Customer=='NASA', 'Rep Code'] = 401
pos.loc[pos.Customer=='DLA', 'Rep Code'] = 401
pos.loc[pos.Customer=='INSULATED WIRE', 'Rep Code'] = 401
pos.loc[pos.Customer=='JDSU', 'Rep Code'] = 401
pos.loc[pos.Customer=='FIRST SOURCE ELECTRONICS', 'Rep Code'] = 401
pos.loc[(pos.Customer=='LOCKHEED MARTIN') & (pos['Item Number'].isin(['SF2960-6939','SF2960-6610','SF2960-6740','SF2960-6619'])), 'Rep Code'] = 401
pos.loc[(pos['Market Region']=='APAC') & (pos['Customer']=='TERADYNE'), 'Rep Code'] = 548
pos.loc[(pos['Market Region']=='APAC') & (pos['Customer']=='TELEDYNE'), 'Rep Code'] = 601
pos.loc[(pos['Market Region']=='EMEA') & (pos['Customer']=='TELEDYNE'), 'Rep Code'] = 701

pos.loc[pos.Customer=='MOUSER', 'Rep Code'] = 401
pos.loc[pos.Customer=='TTI', 'Rep Code'] = 401
pos.loc[pos.Customer=='DIGI-KEY', 'Rep Code'] = 401
pos.loc[pos.Customer=='MICROWAVE COMPONENTS', 'Rep Code'] = 401
pos.loc[pos.Customer=='C.W. SWIFT & ASSOCIATES', 'Rep Code'] = 401
pos.loc[pos.Customer=='APPLIED SPECIALTIES', 'Rep Code'] = 401
pos.loc[pos.Customer=='ARROW ELECTRONICS', 'Rep Code'] = 401
pos.loc[pos.Customer=='A.E. PETSCHE', 'Rep Code'] = 401
pos.loc[pos.Customer=='AVNET', 'Rep Code'] = 401
pos.loc[pos.Customer=='BJG ELECTRONICS', 'Rep Code'] = 401
pos.loc[pos.Customer=='ELECTRO ENTERPRISES', 'Rep Code'] = 401
pos.loc[pos.Customer=='NEWARK', 'Rep Code'] = 401
pos.loc[pos.Customer=='HEILIND', 'Rep Code'] = 401
pos.loc[pos.Customer=='MASTER ELECTRONICS', 'Rep Code'] = 401
pos.loc[pos.Customer=='PEI-GENESIS', 'Rep Code'] = 401
pos.loc[pos.Customer=='RICHARDSON RFPD', 'Rep Code'] = 401
pos.loc[pos.Customer=='SPIRIT ELECTRONICS', 'Rep Code'] = 401
pos.loc[pos.Customer=='FARNELL', 'Rep Code'] = 701
pos.loc[pos.Customer=='KOEHLKE COMPONENTS', 'Rep Code'] = 401
pos.loc[pos.Customer=='CASTLE', 'Rep Code'] = 401
pos.loc[pos.Customer=='SOLLA-TRONICS', 'Rep Code'] = 401
pos.loc[pos.Customer=='RF DEPOT', 'Rep Code'] = 401
pos.loc[pos.Customer=='CPN/DENELEX', 'Rep Code'] = 401
pos.loc[pos.Customer=='POWELL ELECTRONICS', 'Rep Code'] = 401
pos.loc[pos.Customer=='RESCO', 'Rep Code'] = 401
pos.loc[pos.Customer=='JAC INTL', 'Rep Code'] = 401
pos.loc[pos.Customer=='M-RF', 'Rep Code'] = 401

pos.loc[pos.Country=='IL', 'Rep Code'] = 559
pos.loc[pos.Country=='IN', 'Rep Code'] = 630
pos.loc[pos.Country=='TR', 'Rep Code'] = 650

In [ ]:
#clean ZIP codes again
pos['ZIP']=pos['ZIP'].astype(str)
pos.loc[pos['Country']!='US', 'ZIP'] = ''
pos.loc[pos['Country']!='US', 'State'] = ''
pos.loc[pos['ZIP']=='0', 'ZIP'] = ''
pos['lzip']=pos['ZIP'].str.len()
pos.loc[pos.lzip==4, 'ZIP'] = '0'+pos.ZIP

In [ ]:
#assign rep commissions
pos['Commission']=pos['Quantity']*pos['Acquisition Price']*.05
pos.loc[pos['Rep Code'].isin([401,601,610,630,650,701]), 'Commission'] = 0
pos.loc[(pos['Rep Code']==555) & (pos['Item Number'].isin(['7080-0638','7080-0639'])), 'Commission'] = pos['Quantity']*pos['Acquisition Price']*.025
pos.loc[(pos['Rep Code']==548) & (pos['Item Number']=='1211-66317'), 'Commission'] = pos['Commission']*1/2
pos.loc[(pos['Rep Code']==566) & (pos['Item Number']=='SF1211-66198-TD'), 'Commission'] = pos['Commission']*2/3
#pos.loc[(pos['Rep Code']==551) & (pos['Item Number'].isin(['8721-60016','SF1212-6000','SF2991-6002'])) & (pos['Customer']=='PLEXUS'), 'Commission'] = pos['Commission']*1/3

In [ ]:
#view each distributor's POS $ as check to verify nothing changed since last time viewed
pos['Total Resale']=pos['Quantity']*pos['Resale Price']
pos.groupby(['Distributor'])['Total Resale'].sum().reset_index

In [ ]:
#verify total POS $
pos['Total Resale'].sum()

In [ ]:
pos=pos.drop(['Total Resale'], axis = 1)

In [ ]:
#item number clean-up
crosses=pd.read_csv('Crosses.csv').drop_duplicates(subset=['Customer Item'])
pos=pd.merge(pos,crosses, left_on='Item Number',right_on='Customer Item',how='left')
pos.loc[pos['Customer Item'].notnull(), 'Item Number'] = pos['SV Item']
pos=pos.drop(['Customer Item','SV Item'], axis = 1)

In [ ]:
#add some new columns
pos['Commission Status']='Unapproved'
pos['Total Acquisition']=pos['Quantity']*pos['Acquisition Price']
pos['Total Resale']=pos['Quantity']*pos['Resale Price']

In [ ]:
#assign POS ID to match what is in SQL database (pull this file each month)
xref=pd.read_csv('POSCustomerXRef.csv').drop_duplicates(subset=['CustomerName'])
xref=xref[['CustomerName','CustomerID2']]
pos=pd.merge(pos,xref,left_on='Customer',right_on='CustomerName',how='left')
pos.rename(columns={'CustomerID2':'POS Customer ID'},inplace=True)

In [ ]:
#two separate uploads - one for company-wide SQL database, one for backup
pos1=pos[['Date','DistributorID','Distributor','CustomerID','Customer','Purchased','Address','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price','Rep Code','Commission']]
posupload=pos[['Rep Code','Date','DistributorID','Distributor','Original Customer Name','Customer','POS Customer ID','City','State','ZIP','Country','Item Number','Quantity','Acquisition Price','Resale Price','Total Acquisition','Total Resale','Commission','Commission Status']]

In [ ]:
#quickly verify length of table matches the last time length was checked
len(pos1)

In [ ]:
posupload.to_excel('C:/Users/chines/Downloads/September2022POS.xlsx', index = False, header=True)

In [ ]:
pos1.to_excel('C:/Users/chines/Downloads/September2022POS - Excel.xlsx', index = False, header=True)